In [12]:
import re
import os
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Get SQL query file as data input 
def get_sql_file_as_text(file_path):
  
  with open(file_path, 'r') as f:
    sql_content = f.read()
  return sql_content

# Establish connection with Snowflake
conn = snowflake.connector.connect(connection_name="fundingsociety.sg.ap-southeast-1.aws.privatelink")

In [66]:
# Get data from file
# sql_file = "Scratch.sql"
# my_query = get_sql_file_as_text(sql_file)

# Or, type in query manually
ssic_industry_query = "select distinct primary_ssic_description from DEV.SBOX_ADITHYA.SG_GOV_ADDITIONAL_DATA;"

cu_industry_query = "select distinct industry from CBM.CARDUP_DB_REPORTING.COMPANY_DATA;"

# Get data from query
ssic_industry = conn.cursor().execute(ssic_industry_query).fetch_pandas_all()
cu_industry = conn.cursor().execute(cu_industry_query).fetch_pandas_all()

# Manual fixes of industry names

# Remove manufacturing from name
cu_industry['INDUSTRY'] = cu_industry['INDUSTRY'].replace('Electrical/Electronic Manufacturing', 'Electrical and Electronics')

In [67]:
# Sample SSIC df
ssic_industry.head()

,PRIMARY_SSIC_DESCRIPTION
0,MANUFACTURE OF JEWELLERY EXCEPT COSTUME JEWELLERY
1,RETAIL SALE OF TEXTILES
2,MONEY-LENDING
3,GENERAL CLEANING SERVICES (INCLUDING CLEANING ...
4,OTHER BUSINESS SUPPORT SERVICE ACTIVITIES (E.G...


In [68]:
# Sample CU df
cu_industry.head()

,INDUSTRY
0,None
1,"Health, Wellness & Fitness"
2,Food Production
3,Education Management
4,Accounting


In [69]:
# Load a sentence embedding model
model = SentenceTransformer('all-mpnet-base-v2')

In [70]:
# Example!!

# Define L1 and L2 categories
l1_categories = ["Food and Beverages", "Healthcare", "Manufacturing"]
l2_categories = ["Thai Restaurant", "Clinic", "Manufacture of Electronics"]

# Compute embeddings
l1_embeddings = model.encode(l1_categories, convert_to_tensor=True)
l2_embeddings = model.encode(l2_categories, convert_to_tensor=True)

# Create a DataFrame for L2 categories
l2_df = pd.DataFrame({'INDUSTRY': l2_categories})

# Add columns for predicted L1 categories and confidence
l2_df['PREDICTED_L1'] = ""
l2_df['CONFIDENCE'] = 0.0

# Compute cosine similarity and assign predicted L1 and confidence
for i, l2 in enumerate(l2_categories):
    similarities = util.cos_sim(l2_embeddings[i], l1_embeddings)
    best_match_idx = similarities.argmax().item()
    l2_df.loc[i, 'PREDICTED_L1'] = l1_categories[best_match_idx]
    l2_df.loc[i, 'CONFIDENCE'] = similarities[0, best_match_idx].item() 

# Check resulting DF
l2_df.head()

,INDUSTRY,PREDICTED_L1,CONFIDENCE
0,Thai Restaurant,Food and Beverages,0.419048
1,Clinic,Healthcare,0.557442
2,Manufacture of Electronics,Manufacturing,0.594247


In [71]:
# Define L1 and L2 categories
l1_categories = cu_industry['INDUSTRY'].tolist()
l2_categories = ssic_industry['PRIMARY_SSIC_DESCRIPTION'].tolist()

# Compute embeddings
l1_embeddings = model.encode(l1_categories, convert_to_tensor=True)
l2_embeddings = model.encode(l2_categories, convert_to_tensor=True)

# Create a DataFrame for L2 categories
l2_df = pd.DataFrame({'PRIMARY_SSIC_DESCRIPTION': l2_categories})

# Add columns for predicted L1 categories and confidence
l2_df['PREDICTED_L1'] = ""
l2_df['CONFIDENCE'] = 0.0

# Compute cosine similarity and assign predicted L1 and confidence
for i, l2 in enumerate(l2_categories):
    similarities = util.cos_sim(l2_embeddings[i], l1_embeddings)
    best_match_idx = similarities.argmax().item()
    l2_df.loc[i, 'PREDICTED_L1'] = l1_categories[best_match_idx]
    l2_df.loc[i, 'CONFIDENCE'] = similarities[0, best_match_idx].item() 

# Check resulting DF
l2_df.head()

,PRIMARY_SSIC_DESCRIPTION,PREDICTED_L1,CONFIDENCE
0,MANUFACTURE OF JEWELLERY EXCEPT COSTUME JEWELLERY,Luxury Goods & Jewelry,0.608357
1,RETAIL SALE OF TEXTILES,Textiles,0.628690
2,MONEY-LENDING,Banking,0.587641
3,GENERAL CLEANING SERVICES (INCLUDING CLEANING ...,Facilities Services,0.507376
4,OTHER BUSINESS SUPPORT SERVICE ACTIVITIES (E.G...,Consumer Services,0.572760


In [72]:
l2_df.to_csv('l2_df.csv', index=False)